In [237]:
ROOT_DIR = "Music TEST"

In [240]:
from dotenv import load_dotenv
load_dotenv()
import os

def connect_to_nas(username: str, password: str, nas_ip: str):
    conn = SMBConnection(username, password, "local_machine", "nas_device", use_ntlm_v2=True)
    conn.connect(nas_ip)
    print("Connected")
    return conn

conn = connect_to_nas(os.getenv("USER"), os.getenv("PWD"), os.getenv("IP"))

TimeoutError: timed out

In [227]:
from os.path import join, dirname
from smb.SMBConnection import SMBConnection
import logging

logger = logging.getLogger("x")
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(console_handler)

def move_dir(conn: SMBConnection, source: str, destination: str):
    try:
        conn.rename("Public", source, destination)
        print(f"Moved '{source}' to '{destination}'.")
    except Exception as e:
        print(f"Error moving folder: {e}")

def rename_dir(conn: SMBConnection, current_name: str, new_name: str):
    try:
        conn.rename("Public", current_name, new_name)
        print(f"Renamed '{current_name}' to '{new_name}'.")
    except Exception as e:
        print(f"Error renaming folder: {e}")
        
def delete_dir(conn: SMBConnection, folder_path: str):
    try:
        conn.deleteDirectory("Public", folder_path)
        print(f"Deleted '{folder_path}'.")
    except Exception as e:
        print(f"Error deleting folder: {e}")
        
def is_dir_empty(conn: SMBConnection, path: str):
    return all(entry.filename in (".", "..") for entry in conn.listPath("Public", path))

def delete_empty_dir(conn: SMBConnection, path: str):
    if is_dir_empty(conn, path):
        delete_dir(conn, path)
    
def list_empty_dirs_recursively(conn, root_dir) -> list[str]:
    empty_dirs = []

    for path in conn.listPath("Public", root_dir):
        if not path.isDirectory or path.filename in (".", ".."):
            continue
        
        current_dir = join(root_dir, path.filename)
        
        if is_dir_empty(conn, current_dir):
            empty_dirs.append(current_dir)
        else:
            empty_dirs.extend(list_empty_dirs_recursively(conn, current_dir))
    
    return empty_dirs

In [235]:
def delete_all_empty_dirs_recursively(conn, root_dir):
    empty_dirs = list_empty_dirs_recursively(conn, root_dir)
    empty_dirs.sort(key=lambda x: x.count("/"), reverse=True)
    
    for dir_path in empty_dirs:
        delete_dir(conn, dir_path)
        parent_dir = dirname(dir_path)
        while parent_dir != root_dir:
            delete_empty_dir(conn, dir_path)
            parent_dir = dirname(parent_dir)

In [236]:
delete_all_empty_dirs_recursively(conn, ROOT_DIR)

SMBTimeout: 

In [ ]:
import os

ROOT_DIR = "/shares/Public/Music TEST"

def main():
    for i in os.listdir(ROOT_DIR):
        print(i)




if __name__ == "__main__":
    main()
